# IBM Developer Skills Network

# Resnet50 with Keras

# Pretrained model

In [1]:
from keras.preprocessing.image import ImageDataGenerator

In [2]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [4]:
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

In [ ]:
# link https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week3.zip

## Define Global Constants

Here, we will define constants that we will be using throughout the rest of the lab. 

1.  We are obviously dealing with two classes, so _num_classes_ is 2. 
2.  The ResNet50 model was built and trained using images of size (224 x 224). Therefore, we will have to resize our images from (227 x 227) to (224 x 224).
3.  We will training and validating the model using batches of 100 images.

In [5]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

## Construct ImageDataGenerator Instances

In order to instantiate an ImageDataGenerator instance, we will set the **preprocessing_function** argument to _preprocess_input_ which we imported from **keras.applications.resnet50** in order to preprocess our images the same way the images used to train ResNet50 model were processed.


In [22]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [50]:
train_generator = data_generator.flow_from_directory(
                                        r'C:\Users\hp\Downloads\concrete_data_week3\train',
                                        target_size=(image_resize, image_resize),
                                        batch_size = batch_size_training,
                                        class_mode = 'categorical')

      

validation_generator = data_generator.flow_from_directory(
                                        r'C:\Users\hp\Downloads\concrete_data_week3\valid',
                                        target_size=(image_resize, image_resize),
                                        batch_size = batch_size_validation,
                                        class_mode = 'categorical')

Found 30001 images belonging to 2 classes.
Found 10001 images belonging to 2 classes.


Found 10001 images belonging to 2 classes.


# Build Compile and Fit Model

In [42]:
model = Sequential()

Next, we will add the ResNet50 pre-trained model to out model. However, note that we don't want to include the top layer or the output layer of the pre-trained model. We actually want to define our own output layer and train it so that it is optimized for our image dataset. In order to leave out the output layer of the pre-trained model, we will use the argument _include_top_ and set it to **False**.

In [43]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

Then, we will define our output layer as a **Dense** layer, that consists of two nodes and uses the **Softmax** function as the activation function.

In [44]:
# softmax output layer
model.add(Dense(num_classes, activation='softmax'))

In [45]:
model.layers

In [46]:
model.layers[0].layers

Since the ResNet50 model has already been trained, then we want to tell our model not to bother with training the ResNet part, but to train only our dense output layer. To do that, we run the following.

In [47]:
model.layers[0].trainable = False

And now using the _summary_ attribute of the model, we can see how many parameters we will need to optimize in order to train the output layer.

In [48]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 2048)              23587712  
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [49]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Before we are able to start the training process, with an ImageDataGenerator, we will need to define how many steps compose an epoch. Typically, that is the number of images divided by the batch size. Therefore, we define our steps per epoch as follows:

In [51]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

Finally, we are ready to start training our model. Unlike a conventional deep learning training were data is not streamed from a directory, with an ImageDataGenerator where data is augmented in batches, we use the **fit_generator** method.


In [52]:
fit_history = model.fit_generator(
                train_generator,
                steps_per_epoch=steps_per_epoch_training,
                validation_data=validation_generator,
                validation_steps=steps_per_epoch_validation,
                verbose=1,
)

Instructions for updating:
Please use Model.fit, which supports generators.
301/301 [==============================] - 1991s 7s/step - loss: 0.0277 - accuracy: 0.9905 - val_loss: 0.0071 - val_accuracy: 0.9982


Since training can take a long time when building deep learning models, it is always a good idea to save your model once the training is complete if you believe you will be using the model again later. You will be using this model in the next module, so go ahead and save your model.

In [53]:
model.save('classifier_resnet_model.h5')